In [1]:
import pandas as pd
import numpy as np
from glob import glob

In [ ]:
#get list of all data files
data_files = sorted(glob('processed_data/*.csv'))
data_files

In [ ]:
#merge datafiles
data = pd.concat((pd.read_csv(file) for file in data_files), ignore_index=True)

In [ ]:
data.columns

In [ ]:
df = data.drop(['Unnamed: 11', 'station_id', 'Unnamed: 12', 'Unnamed: 13'], axis=1)
df.head(5)

In [ ]:
df.shape

In [ ]:
# Percentage of NAN Values 
missing_values = [(c, df[c].isna().mean()*100) for c in df]
missing_values = pd.DataFrame(missing_values, columns=["column_name", "percentage"])
missing_values

In [ ]:
df.isnull().sum()

In [ ]:
#drop missing values
df.dropna(axis=0, inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
#identify duplicate rows
duplicateRows = df[df.duplicated()]
duplicateRows

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
#save merged dataset
df.to_csv('data/data_merged.csv', index=False)

In [40]:
# read merged data
df_merged =  pd.read_csv('processed_data/data_merged.csv')
df_merged.head()

C:\Users\Azhar\miniconda3\envs\dev\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,ride_length,day_of_week,start_station_id,end_station_id
0,CFA86D4455AA1030,classic_bike,3/16/2021 8:32,3/16/2021 8:36,Humboldt Blvd & Armitage Ave,Stave St & Armitage Ave,casual,0:04:00,3,15651,13266
1,30D9DC61227D1AF3,classic_bike,3/28/2021 1:26,3/28/2021 1:36,Humboldt Blvd & Armitage Ave,Central Park Ave & Bloomingdale Ave,casual,0:10:00,1,15651,18017
2,846D87A15682A284,classic_bike,3/11/2021 21:17,3/11/2021 21:33,Shields Ave & 28th Pl,Halsted St & 35th St,casual,0:16:00,5,15443,13080
3,994D05AA75A168F2,classic_bike,3/11/2021 13:26,3/11/2021 13:55,Winthrop Ave & Lawrence Ave,Broadway & Sheridan Rd,casual,0:29:00,5,13080,13323
4,DF7464FBE92D8308,classic_bike,3/21/2021 9:09,3/21/2021 9:27,Glenwood Ave & Touhy Ave,Chicago Ave & Sheridan Rd,casual,0:18:00,1,525,8


In [41]:
df_merged.dtypes

ride_id               object
rideable_type         object
started_at            object
ended_at              object
start_station_name    object
end_station_name      object
member_casual         object
ride_length           object
day_of_week            int64
start_station_id      object
end_station_id        object
dtype: object

### Change data types

In [44]:
# df_merged['day_of_week'] = pd.to_numeric(df_merged.day_of_week)
# df_merged['started_at'] = pd.to_datetime(df_merged['started_at'], infer_datetime_format=True)
# df_merged['ended_at'] = pd.to_datetime(df_merged['ended_at'], infer_datetime_format=True)
df_merged['ride_duration'] = pd.to_datetime(df_merged.ended_at - df_merged.started_at, infer_datetime_format=True)
df_merged.ride_duration.values.astype('datetime64[D]')

TypeError: dtype timedelta64[ns] cannot be converted to datetime64[ns]

In [4]:
df_merged.describe()

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,ride_length,day_of_week,start_station_id,end_station_id
count,1048575,1048575,1048575,1048575,1048575,1048575,1048575,1048575,1048575,1048575,1048575
unique,1048453,3,140507,140617,688,688,2,11053,15,1547,1517
top,2.02E+15,classic_bike,4/27/2021 17:44,5/22/2021 16:15,Streeter Dr & Grand Ave,Streeter Dr & Grand Ave,member,0:07:00,7,13070,13070
freq,4,740556,59,72,15908,17563,594679,37015,189656,27939,30126


#### Observations
- Three types of rides are available; rideable_type
- Most used bike: classic bike
- 688 distinc start/end stations. But here we also observe a problem.
    - number of stations names must be euqual to number of station_id as each id represents a station. Let's talk a closer look at station id columns

In [6]:
df_merged['start_station_id'].unique

<bound method Series.unique of 0                 15651
1                 15651
2                 15443
3                 13080
4                   525
               ...     
1048570           13137
1048571           13137
1048572           13016
1048573    TA1307000138
1048574    TA1306000010
Name: start_station_id, Length: 1048575, dtype: object>

We see start_station_id col contains alphanumeric values. Let's change them to numeric ids.

In [7]:
df_merged['start_station_id_1'] = df_merged['start_station_id'].str.replace(r'[^\d.]+', '')
df_merged.start_station_id_1

C:\Users\Azhar\miniconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
              ...    
1048570         13137
1048571         13137
1048572         13016
1048573    1307000138
1048574    1306000010
Name: start_station_id_1, Length: 1048575, dtype: object

In [ ]:
# remove alphabets from station_id columns
df['start_station_id'] = df.start_station_id.replace('[^\d.]', '', regex=True).astype(float)